In [ ]:
!pip3 install wget

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !pip install wget
  #!pip install split-folders
  import wget
  import zipfile
  #import split-folders

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.layers import Dense, Flatten, Normalization, Dropout, Conv2D, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom, BatchNormalization, Activation, InputLayer
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras import utils
import os
import matplotlib.pyplot as plt

In [ ]:
import os, warnings
from tensorflow.keras.utils import image_dataset_from_directory

# Project 2 - Veggie Classification

For this assignment you'll need to classify some images of vegetables. 

## Parts

Please do two separate classifications:
<ol>
<li> First, create a model from scratch. 
<li> Use transfer learning to use a pretrained model of your choice, adapted to this data. 
</ol>

There won't be an explicit evaluation of accuracy, but you should take some steps to make each model as accurate as you reasonably can, any tuning option is fair game. Along with that, please structure it into a notebook that is well structured and clear that explains what you did and found. Think about:
<ul>
<li> Sections and headings. 
<li> A description of the approach taken (e.g. what did you do to determine size, tune, evaluate, etc...)
<li> Visualization of some important things such as a confusion matrix and maybe some images. 
<li> Results, mainly focused on the scoring of the test data. 
</ul>

The descriptions and explainations should highlight the choices you made and why you made them. Figure up to about a page or so worth of text total, explain what happened but don't write an essay. 

## Deliverables

Please sumbmit a link to your github, where everyhting is fully run with all the outputs showing on the page. As well, in the notebook please add some kind of switch controlled by a variable that will control if the notebook runs to train the model or to load the model in from weights - so I can download it and click run all, it will load the saved weights, and predict.

### Dataset

The code in the start of this notebook will download and unzip the dataset, and there is also a simple example of creating datasets. You can change the dataset bit to use a different approach if you'd like. The data is already split into train, validation, and test sets. Please treat the separate test set as the final test set, and don't use it for any training or validation. Each folder name is its own label.

### Evaluation

Marking will be based on the following:
<ul>
<li> Models are cretaed, tuned, and effective at classifying the data: 40%
<li> Descriptions and explanations of the approach taken: 20%
<li> Code is well structured and clear: 20%
</ul>

Overall the marking is pretty simple and direct, walk through the process of predicting the veggies, explain what you did, and show the results. If you do that, it'll get a good mark.

### Tips

Some hints that may be helpful to keep in mind:
<ul>
<li> The data is pretty large, so you'll want to use datasets rather than load everything into memory. The Keras docs have a few examples of different ways to load image data, our examples showed image generators and the image from directory datasets.  
<li> Be careful of batch size, you may hit the colab limits. 
<li> You'll want to use checkpoints so you can let it train and pick up where you left off.
<li> When developing, using a smaller dataset sample is a good idea. These weights could also be saved and loaded to jump start training on the full data. 
<li>

### Download and Unzip Data

In [ ]:
# Helper to plot loss
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.legend()
  plt.grid(True)
  plt.show()

def plot_acc(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label='val_accuracy')
  plt.legend()
  plt.grid(True)
  plt.show()

In [ ]:
def bar_custom(current, total, width=80):
    print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))
import wget
import zipfile
zip_name = "train.zip"

url = "https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/vegetable_image_dataset.zip"

if not os.path.exists(zip_name):
    wget.download(url, zip_name, bar=bar_custom)

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
# Generate Datasets - you can change this if desired
# ENSURE FILE PATHS MATCH CORRECTLY
IMAGE_SIZE=(224,224)
train_dir='Vegetable Images/train'
val_dir='Vegetable Images/validation'
batch_size = 16

# Load training data
train_ds = image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    val_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size = batch_size,
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Data Preparation

In [ ]:

BUFFER = 150 
VAL_SPLIT = .2

EPOCHS = 20
SEED = 123

In [ ]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint("logs/weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_accuracy', verbose=2, save_best_only=False, save_weights_only=True, mode='max', save_freq='epoch')

In [ ]:
#callback = tf.keras.callbacks.EarlyStopping(monitor=MONITOR, patience=BASE_PATIENCE, restore_best_weights=True, min_delta=MIN_DELTA, mode=MODE)
#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(WEIGHT_PATH + "weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_accuracy', verbose=2, save_best_only=True, save_weights_only=True, mode='max', save_freq='epoch')


In [ ]:
from tensorflow.keras.layers import Rescaling
rescale = Rescaling(scale=1./255)


In [ ]:
TPAIN = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (rescale(x), y))
#train_ds = train_ds.batch(batch_size)
train_ds = train_ds.shuffle(buffer_size=BUFFER)
train_ds = train_ds.prefetch(buffer_size=TPAIN)

val_ds = val_ds.map(lambda x, y: (rescale(x), y))
#val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(buffer_size=TPAIN)

In [ ]:
#input_shape=IMAGE_SIZE + (3,)
#inputs = keras.Input(shape=input_shape)

## Custom Model Training

In [ ]:
#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model architecture with padding
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(15, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
log_m1 = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=EPOCHS,
      callbacks=[checkpoint_callback]
  )
plot_loss(log_m1)
plot_acc(log_m1)

In [ ]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), padding="same", input_shape=(224, 224, 3)),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3,3), padding="same"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, (3,3), padding="same"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(512, (3,3), padding="same"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, kernel_regularizer="l2"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dropout(.3),
    tf.keras.layers.Dense(128, kernel_regularizer="l2"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(15, activation="softmax")
])


In [ ]:
model1.compile(
  #optimizer=tf.optimizers.Adam(learning_rate=0.001),
  optimizer="rmsprop",
  #loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  loss="categorical_crossentropy",
  metrics=['accuracy']
  )
model1.summary()

In [ ]:
# Train
log_m2 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)
plot_loss(log_m1)
plot_acc(log_m1)

## Transfer Learning Model

## Test Best Models and Illustrate Results

In [ ]:
test_dir='Vegetable Images/test'
test_ds = image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size = batch_size,
)

In [ ]:
if train_mode:
   WEIGHT_PATH = "logs/"


   if os.path.exists(WEIGHT_PATH):# Check if weight files exist
      list_of_files = glob.glob(os.path.join(WEIGHT_PATH, "*.hdf5"))# Get a list of all HDF5 files in the directory
      list_of_files.sort(key=os.path.getctime, reverse=True)# Sort the list of files by creation time (latest file first)
      if list_of_files:
         latest_file = list_of_files[0]# Load the weights from the latest file
         if latest_file:
            model.load_weights(latest_file)